In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

# Preprocessing Data

In [ ]:
#@title Mengambil dataset
import pandas as pd

# Membaca data dari file CSV
id_spreadsheet = "1Y6Mb6VmJjBgcf5_6PZiK7loUcbe91eui" #@param
df = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{id_spreadsheet}/gviz/tq?tqx=out:csv')

In [ ]:
print("Link Spreadsheet")
print(f'https://docs.google.com/spreadsheets/d/{id_spreadsheet}')

In [ ]:
all_col = ["User_Id","Place_Id", "Place_Ratings", "Rating", "Jarak dari Kakbah Mekkah (km)", "Age"]
for col in all_col:
  df[col] = df.apply(lambda row:str(row[col]), axis=1)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Read the data

In [ ]:
ds = tf.data.Dataset.from_tensor_slices(dict(df))

In [ ]:
df.info()

In [ ]:
# ds_feature = ds.map(lambda x: {
#     "Place_Ratings": x["Place_Ratings"],
#     "Age": x["Age"],
#     "Place_Id" : x["Place_Id"]
# })

# ds_feature = ds.map(lambda x: {col : x[col] for i, col in enumerate(all_col)})
ds_feature = ds.map(lambda x: {col : x[col] for i, col in enumerate(["Tempat Wisata", "User_Id"])})
ds_label = ds.map(lambda x: x["Place_Id"])

In [ ]:
ds_feature

In [ ]:
Place_Id = ds.map(lambda x:x["Place_Id"])
Place_Ratings = ds.map(lambda x:x["Place_Ratings"])
Age = ds.map(lambda x:x["Age"])
wisata = ds.map(lambda x:x["Tempat Wisata"])

In [ ]:
wisata

In [ ]:
all_col

# Definisi Model

In [ ]:
#@title Inisiasi Layer StringLookup Yang memungkinkan Input menggunakan string
user_id_vocab = tf.keras.layers.StringLookup(mask_token=None)
place_id_vocab = tf.keras.layers.StringLookup(mask_token=None)
place_rating_vocab = tf.keras.layers.StringLookup(mask_token=None)
rating_vocab = tf.keras.layers.StringLookup(mask_token=None)
distance_vocab = tf.keras.layers.StringLookup(mask_token=None)
age_vocab = tf.keras.layers.StringLookup(mask_token=None)
wisata_vocab = tf.keras.layers.StringLookup(mask_token=None)

user_id_vocab.adapt(ds.map(lambda x: x["User_Id"]))
place_id_vocab.adapt(ds.map(lambda x: x["Place_Id"]))
place_rating_vocab.adapt(ds.map(lambda x: x["Place_Ratings"]))
rating_vocab.adapt(ds.map(lambda x: x["Rating"]))
distance_vocab.adapt(ds.map(lambda x: x["Jarak dari Kakbah Mekkah (km)"]))
age_vocab.adapt(ds.map(lambda x: x["Age"]))
wisata_vocab.adapt(wisata)

In [ ]:
#@title Native Class

# class TourismLensModel(tfrs.Model):
#   # We derive from a custom base class to help reduce boilerplate. Under the hood,
#   # these are still plain Keras Models.

#   def __init__(
#       self,
#       user_id_model: tf.keras.Model,
#       wisata_model: tf.keras.Model,
#       task: tfrs.tasks.Retrieval):
#     super().__init__()

#     # Set up user and movie representations.
#     self.user_id_model = user_id_model
#     self.wisata_model = wisata_model
#     self.task = task

#   def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
#     # Define how the loss is computed.

#     user_id_embeddings = self.user_id_model(features["User_Id"])
#     wisata_embeddings = self.wisata_model(features["Tempat Wisata"])

#     return self.task(user_id_embeddings, wisata_embeddings)

In [ ]:
#@title Definisikan Model TourismLensModel
class TourismLensModel(tfrs.Model):
  def __init__(self):
    super().__init__()

    # Set up user and movie representations.
    self.user_id_model = tf.keras.Sequential([
        wisata_vocab,
        tf.keras.layers.Embedding(wisata_vocab.vocabulary_size(), 64)
    ])
    self.wisata_model = tf.keras.Sequential([
        wisata_vocab,
        tf.keras.layers.Embedding(wisata_vocab.vocabulary_size(), 64)
    ])
    self.task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
        wisata.batch(128).map(wisata_model)
      )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_id_embeddings = self.user_id_model(features["User_Id"])
    wisata_embeddings = self.wisata_model(features["Tempat Wisata"])

    return self.task(user_id_embeddings, wisata_embeddings)

In [ ]:
#@title Defisinikan Beberapa Layer Yang Mungkin Akan Dibutuhkan
wisata_model = tf.keras.Sequential([
    wisata_vocab,
    tf.keras.layers.Embedding(wisata_vocab.vocabulary_size(), 64)
])

user_id_model = tf.keras.Sequential([
    user_id_vocab,
    tf.keras.layers.Embedding(user_id_vocab.vocabulary_size(), 64)
])

place_id_model = tf.keras.Sequential([
    place_id_vocab,
    tf.keras.layers.Embedding(place_id_vocab.vocabulary_size(), 64)
])

place_rating_model = tf.keras.Sequential([
    place_rating_vocab,
    tf.keras.layers.Embedding(place_rating_vocab.vocabulary_size(), 64)
])

rating_model = tf.keras.Sequential([
    rating_vocab,
    tf.keras.layers.Embedding(rating_vocab.vocabulary_size(), 64)
])

distance_model = tf.keras.Sequential([
    distance_vocab,
    tf.keras.layers.Embedding(distance_vocab.vocabulary_size(), 64)
])

age_model = tf.keras.Sequential([
    age_vocab,
    tf.keras.layers.Embedding(age_vocab.vocabulary_size(), 64)
])

# # Define your objectives.
# task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
#     Place_Id.batch(128).map(place_id_model)
#   )
# )

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    wisata.batch(128).map(wisata_model)
  )
)

# Training Model

In [ ]:
#@title Membuat retrieval model.
# model = TourismLensModel(user_id_model, wisata_model , task)
model = TourismLensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ds_feature.batch(4096), epochs=10)

In [ ]:
#@title Brute Force Layer Untuk Set Up/Mengatur retrieval menggunakan representasi terlatih
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_id_model)
index.index_from_dataset(
    wisata.batch(100).map(lambda title: (title, model.wisata_model(title))))

# Prediksi Model

In [ ]:
#@title Fungsi Untuk Meratakan list bercabang
def flatten_list(nested_list ):
  return [item for sublist in nested_list for item in sublist]

In [ ]:
#@title Prediksi model rekomendasi
_, titles = index(np.array(["1"]), k=50)
print(f"Top 50 recommendations for user 42: {flatten_list(titles)}")

In [ ]:
titles_ = flatten_list(titles)
titles_ = [_titles_.numpy().decode('utf-8') for _titles_ in titles_]
print(titles_)

In [ ]:
import pandas as pd

df_result_rekomendasi = pd.DataFrame({
    'Nama Wisata' : titles_
})

In [ ]:
#@title Hasil Rekomendasi
df_result_rekomendasi

In [ ]:
df_result_rekomendasi.to_csv("HasilRekomendasiWisataMekkah.csv", index=False)

In [ ]:
len(flatten_list(titles))

# Export Model

In [ ]:
#@title Export Model
model.save("recommender_model_of_wisata_mekkah.keras")

In [ ]:
# #@title Export Model
# index.save("recommender_model_of_wisata_mekkah")

In [ ]:
# #@title Export Ke TFLITE
# # Convert the model
# converter = tf.lite.TFLiteConverter.from_keras_model("recommender_model_of_wisata_mekkah.keras") # path to the SavedModel directory
# tflite_model = converter.convert()

# # Save the model.
# with open('model.tflite', 'wb') as f:
#   f.write(tflite_model)

In [ ]:
#@title Export Weight Model
# @markdown Sumber Cara export/save weight serta load weight ada di link https://stackoverflow.com/questions/73659720/how-to-save-tensorflow-recommenders-framework-model
model.save_weights('recommender_weights_of_wisata_mekkah', save_format='tf')

# Contoh Implementasi menjadi RestAPI menggunakan FastAPI dan NGROK

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn flask-ngrok ngrok flask python-multipart aiofiles -q

**Catatan Untuk FastAPI dan NGROK**

ketika kode berikut dijalankan akan menghasilkan Public URL, Public URL ini akan selalu berganti jika kode berikut dijalankan lagi(berhenti lalu dijalankan lagi). Untuk kode ngrok_auth_token berikut bisa diganti dengan yang tersedia, bisa dapatkan ngrok_auth_token di website officialnya dan sign up.

Public URL ini bisa digunakan sebagai restAPI, contoh penggunaan nya

https://c8fd-35-197-89-51.ngrok-free.app/recommender?id_input=3


In [ ]:
from fastapi import FastAPI
from fastapi import File, UploadFile
from PIL import Image
from io import BytesIO
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from fastapi.responses import FileResponse
import shutil
from fastapi.staticfiles import StaticFiles

ngrok_auth_token = "1TfbVOS48SeXdQ7rJ2do5JjJFxG_4d5K3jMerctfbUsXvidrT" #@param
ngrok.set_auth_token(ngrok_auth_token)

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get("/")
async def root():
    """
    Fungsi ini hanyalan untuk test

    Method : Get

    Output :
    Message(String) : Pesan Yang dihasilkan
    """
    return {
        "message" : "Selamat Datang Di Rest API Sistem Rekomendasi Wisata Mekkah"
    }

@app.get("/recommender")
async def recommender(id_input: str = None):
    """
    Fungsi ini digunakan untuk rekomendasi dalam restAPI

    Method : Get

    Parameter :

    id_input(String) : Input Data berupa id string

    Output :

    message(String) : pesan yang dihasilkan
    tempat_wisata(List(String)) : list rekomendasi tempat wisata

    """
    try:
      _, tempat_wisata = index(np.array([id_input]), k=50)
      tempat_wisata = flatten_list(tempat_wisata)
      tempat_wisata = [str(wisata_.numpy().decode('utf-8')) for wisata_ in tempat_wisata]
      # print(f"Top 50 recommendations for user 42: {flatten_list(titles)}")
      return {
          "message" : "Hasil Rekomendasi",
          "tempat_wisata" : tempat_wisata
      }
    except:
      return {
          "message" : "Predict Failed"
      }

listener = ngrok.connect(8000)
# listener = ngrok.forward(8000, authtoken_from_env=True)
print('Public URL:', listener.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)


# Catatan Tambahan

**Saran**

Untuk Implementasi Model sebaiknya dibuat restAPI misalnya menggunakan FastAPI menggunakan bahasa python pada kode ini(sebaiknya dibuat kode file python, bukan format jupyter notebook atau google colab) mengingat export model dan export weight model jauh lebih kompleks dari tensorflow yang biasanya. RestAPI ini nantinya bisa dijalankan di VirtualMachine(VM) atau Cloud(bisa explore sendiri bagaimana cara menjalankan file python di cloud dan set up API nya). Setelah sudah jalan, RestAPI ini bisa dilakukan pengiriman request dari mobile(android) ke program restAPI untuk mendapatkan hasil